**Table of contents**<a id='toc0_'></a>    
- [Create a Meal Using Signatures](#toc1_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[Create a Meal Using Signatures](#toc0_)

Playing around with signatures by doing a simple meal suggestion

Learning to build my domain models for usage later on.

Domain: 

* Meal  
    * Dish

In [68]:
import dspy 
from pydantic import BaseModel
from typing import Literal

from common.my_settings import MySettings  
from common.utils import md
from common.llm_client_factory import LlmClientFactory
from dspy_utils.dspy_helpers import md_dspy

settings = MySettings().get()
lm = dspy.LM('gpt-4.1', temperature=0.8, model_type='chat', cache=False, api_key=settings.OPENAI_API_KEY)
dspy.configure(lm=lm)

Getting keys from environment variables


In [69]:
class Dish(BaseModel):
    # Output
    
    dish_type: Literal["Fish", "Vegetarian", "Chicken"] = dspy.OutputField() # This field is locked to a range of the Literal
    name: str = dspy.OutputField()
    
class Meal(dspy.Signature):
    """
    Suggest a meal
    """
    
    # Inputs
    preferred_day = dspy.InputField(desc="Preferred day of the week for the meal")
    
    # Outputs
    dish: Dish = dspy.OutputField()
    day_of_the_week: Literal["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"] = dspy.OutputField()
    
    # Display
    def __str__(self):
        return f"Meal for {self.preferred_day}: {self.dish} on {self.day_of_the_week}"


In [ ]:
# Call LLM
predict = dspy.Predict(Meal)

# Play around with preferred_day value like 
# setting: "weekday", "28 Aug 2025", "weekend" and see how it changes "day_of_the_week"
result = predict(preferred_day="When the Wimbeldon finals are on") # Should choose Sunday
# result = predict(preferred_day="28 Aug 2025") # Should select a "Thursday"
# result = predict(preferred_day="Weekend")  # Should select Sat or Sun
# result = predict(preferred_day="Weekday")  # Should select a work day

md(result)

Prediction(
    dish=Dish(dish_type='Chicken', name='Coronation Chicken Sandwiches'),
    day_of_the_week='Sunday'
)